In [ ]:
!nvidia-smi

Tue Jun 13 14:47:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! pip install -q langchain transformers sentence_transformers llama-index


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.0/506.0 kB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 9.4 MB/s eta 0:00:00
 

In [ ]:
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex,GPTVectorStoreIndex, PromptHelper
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LLMPredictor, ServiceContext
import torch
from langchain.llms.base import LLM
from transformers import pipeline

In [ ]:
class customLLM(LLM):
    model_name = "google/flan-t5-large"
    pipeline = pipeline("text2text-generation", model=model_name, device=0, model_kwargs={"torch_dtype":torch.bfloat16})

    def _call(self, prompt, stop=None):
        return self.pipeline(prompt, max_length=9999)[0]["generated_text"]
    @property
    def _identifying_params(self):
        return {"name_of_model": self.model_name}
    @property
    def _llm_type(self):
        return "custom"


llm_predictor = LLMPredictor(llm=customLLM())


In [ ]:
hfemb = HuggingFaceEmbeddings()
embed_model = LangchainEmbedding(hfemb)


In [ ]:
# https://www.sunlife.ca/en/health/personal-health-insurance/basic-plan/
text1 = """

Basic healthcare plan:

Prescription drugs
Pay-direct drug card (except in Quebec)
60% reimbursement
No deductible
$750 annual maximum of eligible expenses can be claimed
Up to $5 paid towards dispensing fee on prescriptions
Includes smoking cessation drugs ($250 lifetime maximum)
Excludes:
contraceptives
fertility drugs
dietary supplements, vitamins
over-the-counter medications, even if prescribed by a physician or nurse practitioner
drugs to treat obesity


Preventive dental care
60% reimbursement
$500 annual maximum
Recall visits every 9 months
3-month waiting period before coverage begins
Preventive dental coverage includes:
Examinations and diagnosis
Tests, x-rays and lab exams
White fillings
Space maintainers for children under 12 years of age
Scaling, minor extractions
Pit and fissure sealant for children under 19 years of age

Hearing aids
60% reimbursement
$400 maximum every 5 years

Accidental dental
60% reimbursement
$2,000 per fracture or injury


Ambulance
60% reimbursement
Ground or air ambulance services

Medical equipment and services
60% reimbursement
$2,500 annual maximum combined with in-home nursing and home care
$20,000 lifetime maximum combined with in-home nursing and home care
The following items are covered to a maximum of $2,500 per year and $20,000 lifetime combined with in-home nursing and home care:

Artificial limbs or other prosthetic appliances
Orthopaedic shoes and orthotics, limited to a maximum $150 in a calendar year
Splints and crutches
Casts
Braces
Wigs and hairpieces, limited to a maximum $100 in a calendar year
Oxygen
Continuous glucose monitors
Diagnostic laboratory tests and x-ray examinations
Blood glucose monitor, limited to $150 per 5-year period
Rental, or purchase at our option, of durable equipment required for use in the patient's home and that we have approved. Eligible durable equipment includes, but is not limited to, items such as:
wheelchairs ($1,000 lifetime maximum)
walkers
hospital beds
traction kits
For Quebec residents only: medically necessary MRIs, ultrasounds, CAT and CT scans to a combined maximum of $750 in a calendar year

Private duty nursing*
60% reimbursement
$2,500 annual maximum combined with medical equipment and services
$20,000 lifetime maximum combined with medical equipment and services
*Private duty nursing includes services of registered nurses, registered practical nurses or registered nursing assistants.

Paramedical practitioners
60% reimbursement
$25 maximum per visit
Up to $250 per year, per type of practitioner
Increased coverage for psychology and social worker services to a shared maximum of $500 per year, $35 maximum per visit
Paramedical practitioners include: acupuncturists, chiropractors, naturopaths, osteopaths, physiotherapists, podiatrists, chiropodists, psychologists, social workers, registered massage therapists, speech language pathologists
Vision care
No coverage. Vision care is available on the Standard and Enhanced plan
Emergency travel medical
No coverage. Emergency travel medical coverage is available on the Standard and Enhanced plan
Semi-private hospital room
This is an optional benefit with this plan and includes:

85% reimbursement
Coverage up to $200 daily and $5,000 annually
Convalescent hospital:* $20 per day up to 180 days per incident
If you are pregnant at the time your application, we will pay for up to 2 days of hospitalization due to pregnancy.

*Convalescent hospitals are facilities licensed to provide convalescent care and treatment for sick or injured patients on an in-patient basis.

Additional services
Access to mysunlife.ca and the my Sun Life Mobile app lets you manage your personal health insurance plan online.
Age requirements
Coverage is available for new applicants age 69 and under and is renewable for customers over 70.
You and everyone included in your application must be age 69 or under on the date you apply.
Eligibility
You are eligible for coverage if:

You are in Canada when you complete and submit the application
You and your family members are residents of Canada and have provincial or territorial health and drug coverage
Quebec residents must also have and continue to have group drug coverage provided by an employer, through membership in an order or association or through the Régie de l'assurance maladie du Québec (RAMQ). A person not covered under a group benefit plan or through RAMQ is not eligible for coverage.
Your dependent children are under age 25 (dependent children aged 21 to 24 must be full-time students)
You are not eligible for coverage if:

You had or have any of these illnesses or conditions
You are currently awaiting doctor recommended tests or investigations
You have a pending surgery
More information
You must also apply to your provincial or territorial government health and drug plan first, then submit a claim to us for the unpaid portion.
All annual or lifetime maximums are per person and based on a calendar year (January 1 to December 31).
We may change rates each year. If we do change the rates, we'll send you written notice at least 30 days before the change.
Waiting periods begin on the date the policy becomes effective.
This page does not form part of your policy. If there are any inconsistencies between the content on this page and your policy, the terms of your policy will apply. Your policy includes exclusions and limitations of coverage.
Find more exclusions and limitations in the Basic plan with semi-private hospital room coverage sample policy.


Standard healthcare plan:

Plan details:
Prescription drugs
Pay-direct drug card (except Quebec)
70% reimbursement for the first $7,000 of annual eligible expenses (that means up to $4,900 of expenses will be paid), 100% reimbursement for the next $93,000 of annual eligible expenses
$100,000 annual maximum of eligible expenses can be claimed
No deductible
Includes smoking cessation drugs ($250 lifetime maximum)
Excludes:
contraceptives
fertility drugs
dietary supplements, vitamins
over-the-counter medications, even if prescribed by a physician or nurse practitioner
drugs to treat obesity
Dental
This is an optional benefit with this plan and includes:

Preventive dental care
70% reimbursement
$750 annual maximum
Recall visits every 9 months
3-month waiting period before coverage begins
Preventive dental coverage includes:
Examinations and diagnosis
Tests, x-rays and lab exams
White fillings
Space maintainers for children under 12 years of age
Scaling, minor extractions
Pit and fissure sealant for children under 19 years of age

Restorative dental care
No coverage. Restorative dental care is available on the Enhanced plan as an optional benefit.

Orthodontics
No coverage. Orthodontics is available on the Enhanced plan as an optional benefit.

Hearing aids
100% reimbursement
$500 maximum every 5 years


Accidental dental
100% reimbursement
Limited to $2,000 per fracture or injury

Ambulance
100% reimbursement
Ground or air ambulance services


Medical equipment and services
100% reimbursement
$5,000 annual maximum
The following items have a combined $5,000 annual maximum:

Orthopaedic shoes and orthotics, limited to $225 in a calendar year
Splints and crutches limited to $500 in a calendar year
Casts
Braces
Wigs and hairpieces, limited to $500 lifetime maximum
Oxygen
Continuous glucose monitors
Diagnostic laboratory and x-ray examinations
Blood glucose monitor, limited to $300 during a 5-year period
Rental, or purchase at our option, of durable equipment required for use in the patient's home and is approved by us. Eligible durable equipment includes, but is not limited to, items such as:
Wheelchairs, to a $4,000 lifetime maximum
walkers
hospital beds, to a maximum of $1,500 lifetime
traction kits
artificial limbs or other prosthetic appliances, breast prostheses are limited to $200 in a calendar year
For Quebec residents only: medically necessary MRIs, ultrasounds, CAT and CT scans


Private duty nursing*
100% reimbursement
Combined $5,000 annual maximum and combined $25,000 lifetime maximum
*Private duty nursing includes services of registered nurses, registered practical nurses or registered nursing assistants

Paramedical practitioners
100% reimbursement
No per-visit maximum
Up to $300 per year, per type of practitioner
Increased coverage for psychology and social worker services to a shared maximum of $1,000 per year
Paramedical practitioners include: acupuncturists, chiropractors, naturopaths, osteopaths, physiotherapists, podiatrists, chiropodists, psychologists, social workers, registered massage therapists, speech language pathologists
Vision care
100% reimbursement
$250 maximum every 2 calendar years, including $50 maximum per year per eye exam
1-year waiting period before coverage begins
Includes coverage for: prescription eyeglasses, prescription contact lenses, prescription sunglasses and laser eye surgery
Emergency travel medical
100% reimbursement
$1 million lifetime maximum
Coverage provided for the first 60 days of a trip
Covers travel outside of your province or outside of Canada
Available until the age of 80
If you have a pre-existing medical condition where symptoms have appeared or required medical attention, hospitalization or treatment (this includes changes in medication or dosage), and existed during the 9 months before your trip, expenses related to this condition are not included.
Semi-private hospital room
This is an optional benefit with this plan and includes:

85% reimbursement
Coverage up to $200 daily and $5,000 annually
Convalescent hospital:* $20 per day up to 180 days per incident
If you are pregnant at the time of application, we will pay for up to 2 days of hospitalization due to pregnancy.

*Convalescent hospitals (facility licensed to provide convalescent care and treatment for sick or injured patients on an in-patient basis)

Additional services
Access to mysunlife.ca and the my Sun Life Mobile app lets you manage your personal health insurance plan online.
Age requirements
Coverage is available for new applicants age 69 and under and is renewable for customers over 70.
You and everyone included in your application must be age 69 or under on the date you apply.
Eligibility
You are eligible for coverage if:

You are in Canada when you complete and submit the application
You and your family members are residents of Canada and have provincial or territorial health and drug coverage
Quebec residents must also have and continue to have group drug coverage provided by an employer, through membership in an order or association or through the Régie de l'assurance maladie du Québec (RAMQ). A person not covered under a group benefit plan or through RAMQ is not eligible for coverage.
Your dependent children are under age 25 (dependent children aged 21 to 24 must be full-time students)
You are not eligible for coverage if:

You had or have any of these illnesses or conditions
You are currently awaiting doctor recommended tests or investigations
You have a pending surgery
More information
You must also apply to your provincial or territorial government health and drug plan first, then submit a claim to us for the unpaid portion.
All annual or lifetime maximums are per person and based on a calendar year (January 1 to December 31).
We may change rates each year. If we do change the rates, we'll send you written notice at least 30 days before the change.
Waiting periods begin on the date the policy becomes effective.
This page does not form part of your policy. If there are any inconsistencies between the content on this page and your policy, the terms of your policy will apply. Your policy includes exclusions and limitations of coverage.


Enhanced healthcare plan:


Plan details:
Prescription drugs
Pay-direct drug card (except Quebec)
80% reimbursement for the first $5,000 of annual eligible expenses (that means up to $4,000 of expenses will be paid), 100% reimbursement for the next $245,000 of annual eligible expenses
$250,000 annual maximum of eligible expenses can be claimed
No deductible
Includes smoking cessation drugs ($250 lifetime maximum)
Includes oral contraceptives
Excludes:
contraceptives (other than oral)
fertility drugs
dietary supplements, vitamins
over-the-counter medications, even if prescribed by a physician or nurse practitioner
drugs to treat obesity

Preventive dental care
80% reimbursement
$750 annual maximum
Recall visits every 9 months
3-month waiting period before coverage begins
Preventive dental coverage includes:
Examinations and diagnosis
Tests, x-rays and lab exams
White fillings
Space maintainers for children under 12 years of age
Scaling, minor extractions
Pit and fissure sealant for children under 19 years of age

Restorative dental care
50% reimbursement
$500 annual maximum
1-year waiting period before coverage begins
Restorative dental coverage includes:
Endodontics
Periodontics
Oral surgery
Crowns
Onlays
Bridges
Dentures (and repairs to dentures)

Orthodontics
60% reimbursement
$1,500 lifetime maximum
2-year waiting period before coverage begins
Orthodontic coverage includes:
Interceptive, interventive or preventive orthodontic services other than space maintainers
Comprehensive orthodontic treatment, using a removable or fixed appliance or combination of both. This includes diagnostic procedures, formal treatment and retention

Hearing aids
100% reimbursement
$600 maximum every 5 years

Accidental dental
100% reimbursement
$2,000 per fracture or injury

Ambulance
100% reimbursement
Ground or air ambulance services

Medical equipment and services
100% reimbursement
$5,000 annual maximum
The following items have a combined $5,000 annual maximum:

Orthopaedic shoes and orthotics, limited to $250 in a calendar year
Splints and crutches, limited to $500 in a calendar year
Casts
Braces
Wigs and hairpieces, limited to a lifetime maximum of $500
Oxygen
Continuous glucose monitors
Diagnostic laboratory tests and x-ray examinations
Blood glucose monitor, limited to $300 per each 5-year period
Rental, or purchase at our option, of durable equipment required for use in the patient's home and is approved by us. Eligible durable equipment includes, but is not limited to, items such as:
wheelchairs, to a $4,000 lifetime maximum
walkers
hospital beds, to a $1,500 lifetime maximum
traction kits
artificial limbs or other prosthetic appliances, breast prostheses up to $200 per year
For Quebec residents only, medically necessary MRIs, ultrasounds, CAT and CT scans

Private duty nursing*
100% reimbursement
Combined $10,000 annual maximum and combined $30,000 lifetime maximum
*Private duty nursing includes services of registered nurses, registered practical nurses or registered nursing assistants.


Paramedical practitioners
100% reimbursement
No per-visit maximum
Up to $400 per year, per type of practitioner
Increased coverage for psychology and social worker services to a shared maximum of $1500 per year
Paramedical practitioners include: acupuncturists, chiropractors, naturopaths, osteopaths, physiotherapists, podiatrists, chiropodists, psychologists, social workers, registered massage therapists, speech language pathologists
Vision care
100% reimbursement
$300 maximum every 2 calendar years, includes $50 maximum per year per eye exam
1-year waiting period before coverage begins
Includes coverage for: prescription eyeglasses, prescription contact lenses, prescription sunglasses and laser eye surgery
Emergency travel medical
100% reimbursement
$1 million lifetime maximum
Coverage provided for the first 60 days of a trip
Covers travel outside of your province or outside of Canada
Available until the age of 80
If you have a pre-existing medical condition where symptoms have appeared or required medical attention, hospitalization or treatment (this includes changes in medication or dosage), and existed during the 9 months before your trip, expenses related to this condition are not included.
Semi-private hospital room
This is an optional benefit with this plan and includes:

85% reimbursement
Coverage up to $200 daily and $5,000 annually
Convalescent hospital:* $20 per day up to 180 days per incident
If you are pregnant at the time of application, we will pay for up to 2 days of hospitalization due to pregnancy.

*Convalescent hospitals are facilities licensed to provide convalescent care and treatment for sick or injured patients on an in-patient basis.

Additional services
Access to mysunlife.ca and the my Sun Life Mobile app lets you manage your personal health insurance plan online.
Age requirements
Coverage is available for new applicants age 69 and under and is renewable for customers over 70.
You and everyone included in your application must be age 69 or under on the date you apply.
Eligibility
You are eligible for coverage if:

You are in Canada when you complete and submit the application
You and your family members are residents of Canada and have provincial or territorial health and drug coverage
Quebec residents must also have and continue to have group drug coverage provided by an employer, through membership in an order or association or through the Régie de l'assurance maladie du Québec (RAMQ). A person not covered under a group benefit plan or through RAMQ is not eligible for coverage
Your dependent children are under age 25 (dependent children aged 21 to 24 must be full-time students)
You are not eligible for coverage if:

You had or have any of these illnesses or conditions
You are currently awaiting doctor recommended tests or investigations
You have a pending surgery
More information
You must also apply to your provincial or territorial government health and drug plan first, then submit a claim to us for the unpaid portion.
All annual or lifetime maximums are per person and based on a calendar year (January 1 to December 31).
We may change rates each year. If we do change the rates, we'll send you written notice at least 30 days before the change.
Waiting periods begin on the date the policy becomes effective.
This page does not form part of your policy. If there are any inconsistencies between the content on this page and your policy, the terms of your policy will apply. Your policy includes exclusions and limitations of coverage.



Sample question:

What is the coverage for prescription drugs in my healthcare plan?

The coverage for prescription drugs varies based on the plan. For example, in the Basic Healthcare Plan, you would receive a 60% reimbursement with a $750 annual maximum. In the Standard Healthcare Plan, it provides 70% reimbursement for the first $7,000 of annual eligible expenses and 100% reimbursement for the next $93,000. In the Enhanced Healthcare Plan, it offers 80% reimbursement for the first $5,000 of annual eligible expenses and 100% reimbursement for the next $245,000.
Is dental care covered in my healthcare plan?

Dental care coverage is available as an optional benefit in the Standard Healthcare Plan and is included in the Enhanced Healthcare Plan. The coverage may include preventive and restorative dental care, and orthodontics in the Enhanced Healthcare Plan.
How much coverage do I have for hearing aids?

The coverage for hearing aids depends on the plan. In the Basic Healthcare Plan, you would receive a 60% reimbursement with a $400 maximum every 5 years. In the Standard Healthcare Plan, it offers 100% reimbursement with a $500 maximum every 5 years. In the Enhanced Healthcare Plan, it provides 100% reimbursement with a $600 maximum every 5 years.
What is the coverage for ambulance services?

Ambulance service coverage varies based on the plan. In the Basic Healthcare Plan, it offers a 60% reimbursement for ground or air ambulance services. In the Standard and Enhanced Healthcare Plans, it provides 100% reimbursement for ground or air ambulance services.
Are paramedical practitioners covered in my healthcare plan?

Paramedical practitioner coverage is available in all three plans. The reimbursement percentage and limits may vary. For example, in the Basic Healthcare Plan, it offers a 60% reimbursement with a $25 maximum per visit, up to $250 per year per type of practitioner. In the Standard and Enhanced Healthcare Plans, it provides 100% reimbursement with no per-visit maximum, up to $300 per year per type of practitioner in the Standard plan, and up to $400 per year per type of practitioner in the Enhanced plan.
What is the coverage for vision care in my healthcare plan?

Vision care coverage varies based on the plan. In the Basic Healthcare Plan, there is no coverage for vision care. In the Standard Healthcare Plan, it offers 100% reimbursement with a $250 maximum every 2 calendar years. In the Enhanced Healthcare Plan, it provides 100% reimbursement with coverage for prescription eyeglasses, contact lenses, sunglasses, and laser eye surgery.
Is emergency travel medical coverage included in my healthcare plan?

Emergency travel medical coverage is not available in the Basic Healthcare Plan. However, it is included in both the Standard and Enhanced Healthcare Plans. The coverage offers 100% reimbursement with a $1 million lifetime maximum.
Can I choose a semi-private hospital room?

The option for a semi-private hospital room is available as an add-on benefit in all three plans. The coverage may vary. For example, in the Basic Healthcare Plan, it is an optional benefit with 85% reimbursement, coverage up to $200 daily and $5,000 annually. The same coverage details apply to the Standard and Enhanced Healthcare Plans.

"""

In [ ]:
#pip install PyPDF2

In [ ]:
# # importing required modules
# from PyPDF2 import PdfReader

# # creating a pdf reader object
# reader = PdfReader('sbc-sample.pdf')

# # Initialize an empty string to store the combined text
# combined_text = ""

# # Iterate over all pages
# for page in reader.pages:
#     # Extracting text from each page
#     text = page.extract_text()
#     # Append the text to the combined_text string
#     combined_text += text

# # Print the combined text
# print(combined_text)

In [ ]:
#documents = SimpleDirectoryReader('data').load_data()

from llama_index import Document

text_list = [text1]

documents = [Document(t) for t in text_list]

In [ ]:
# set number of output tokens
num_output = 500
# set maximum input size
max_input_size = 512
# set maximum chunk overlap
max_chunk_overlap = 1


prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)


In [ ]:
#index = GPTSimpleVectorIndex(documents, embed_model=embed_model, llm_predictor=llm_predictor)

#index = GPTListIndex.from_documents(documents, embed_model=embed_model, llm_predictor=llm_predictor)

#index.save_to_disk('index.json')

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model)
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)


In [ ]:
import logging

logging.getLogger().setLevel(logging.CRITICAL)

In [ ]:
#response = index.query( "What's the cost of Whisper model?")


In [ ]:
query_engine = index.as_query_engine()
prompt = "What is the coverage for prescription drugs in my basic healthcare plan?"
response = query_engine.query(prompt)
print(response)


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


60% reimbursement
